In [1571]:
#Import libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rapidfuzz
import difflib
from nltk.corpus import stopwords
from yake import KeywordExtractor

In [1572]:
Stops = stopwords.words('english') #Returns all stopwords in the English language. 
print(Stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Conneticut

In [1573]:
State = 'CT'
AllOccs = pd.read_csv('../../../Data/Post1790/aggregated_CD_noname.csv') #Returns aggregated CSV file. 
AllOccs.drop(columns=['Unnamed: 0'], inplace=True) #Drop duplicate index column which appears.
AllOccsCT = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsCT = AllOccsCT.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsCT)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       1064.0      75.0          532.0          37.0      508.0      51.0   
1        449.0      96.0          224.0          97.0      232.0      10.0   
2        196.0      75.0           98.0          37.0      172.0      24.0   
3         53.0      58.0           26.0          79.0       67.0       6.0   
4        121.0      22.0           60.0          61.0       32.0      72.0   
..         ...       ...            ...           ...        ...       ...   
478       39.0      11.0           19.0          55.0       14.0       7.0   
479       13.0      21.0            6.0          61.0        4.0      74.0   
480      533.0      34.0          266.0          66.0      446.0      66.0   
481      658.0      47.0          329.0          23.0      558.0      50.0   
482       12.0      22.0            6.0          11.0       11.0       NaN   

           town state occupation             county name_type  

In [1574]:
OccsTotalsCT = AllOccsCT.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsCT.to_string())

                                                           occupation  6p_total
0                                   Adminastrator of the Asa Waterman    218.47
1                                      Adminastrator to David Fellows      0.00
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased     61.30
3                        Adminastrator to the Estate of Samuel Tilley     94.67
4            Adminastrator to the estate of Uriah Loomis Jun Deceased     73.50
5                                          Administrator to J McCurdy   3413.72
6                         Administrator to the estate of Daniel Unkus     54.05
7                Adminstrator to the Estate of Ezekiel Carey Deceased     77.61
8                  Adminstrator to the Estate of Ruth Skiffe Deceased     87.74
9                                                          Apothecary      7.72
10                                                           Attorney   4863.67
11                                      

In [1575]:
#Function that returns the same dataframe, but with all stopwords removed from the occupation titles. 
def RemoveStopwords(OccsTotals):
    OccsTotals['occupation'] = OccsTotals['occupation'].apply(lambda OccName: ' '.join(word.lower() for word in OccName.split() if word not in Stops))
    return OccsTotals 

In [1576]:
OccsTotalsCT = RemoveStopwords(OccsTotalsCT) #Call 
print(OccsTotalsCT.to_string())

                                              occupation  6p_total
0                             adminastrator asa waterman    218.47
1                            adminastrator david fellows      0.00
2   adminastrator estate epaphras bull hartford deceased     61.30
3                     adminastrator estate samuel tilley     94.67
4         adminastrator estate uriah loomis jun deceased     73.50
5                                administrator j mccurdy   3413.72
6                      administrator estate daniel unkus     54.05
7             adminstrator estate ezekiel carey deceased     77.61
8               adminstrator estate ruth skiffe deceased     87.74
9                                             apothecary      7.72
10                                              attorney   4863.67
11                                          attorney law   6250.93
12                                            blacksmith    724.71
13                                                broker    20

In [1577]:
#This function will help determine how many words we can acceptably cut off from the end. 
def FindRemoveConst(Occupations):
    TempMaxLength = 2
    
    print('Occupation Before --> Occupation After')
    print('--------------------------------------')
    print('\n')
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > TempMaxLength): #If occupation has more than two words, remove last two words.
            OccupationAfter = Occupation.rsplit(' ', Remove)[0] 
        else:
            OccupationAfter = Occupation #Else, set them both equal to each other; nothing changes. 
        
        print(Occupation + ' --> ' + OccupationAfter)

In [1578]:
FindRemoveConst(OccsTotalsCT['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


adminastrator asa waterman --> adminastrator asa
adminastrator david fellows --> adminastrator david
adminastrator estate epaphras bull hartford deceased --> adminastrator estate epaphras bull hartford
adminastrator estate samuel tilley --> adminastrator estate samuel
adminastrator estate uriah loomis jun deceased --> adminastrator estate uriah loomis jun
administrator j mccurdy --> administrator j
administrator estate daniel unkus --> administrator estate daniel
adminstrator estate ezekiel carey deceased --> adminstrator estate ezekiel carey
adminstrator estate ruth skiffe deceased --> adminstrator estate ruth skiffe
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney law
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier hartford bank --> cashier hartford
clothier --> clothier
committee 1st ecclestical society --> committee 1st ecclestical
committee 1st socie

In [1579]:
#Declare necessary parameters for KeywordExtractor() function.
MaxLength = 2
Remove = 2
language = 'en' #Set language to English.
MaxSize = 1  #Each keyword can only be a max. length of 1.
DupThres = 0.5 #Words can not be used repeatedly in keywords. Treat each word separately. 
KeyExtractor = KeywordExtractor(lan=language, n=MaxSize, dedupLim=DupThres, features=None) #Create KeywordExtractor object.

In [1580]:
#Print what the merged occupations will look like. This will help us find any exceptions. 
def PrintFutureOccupations(Occupations):    
    print('Occupations --> Potential Cleaned Occupations')
    print('---------------------------------------------')
    print('\n')
        
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
                    
        print(Occupation + ' --> ' + str(Keywords[0][0])) #Print current occupation title to hypothetical cleaned occupation title.

In [1581]:
PrintFutureOccupations(OccsTotalsCT['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
adminastrator --> adminastrator
adminastrator estate epaphras bull --> adminastrator
adminastrator estate --> adminastrator
adminastrator estate uriah loomis --> adminastrator
administrator --> administrator
administrator estate --> administrator
adminstrator estate ezekiel --> adminstrator
adminstrator estate ruth --> adminstrator
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier --> cashier
clothier --> clothier
committee 1st --> committee
committee 1st --> committee
cooper --> cooper
doctor --> doctor
esquire --> esquire
executor ezra selden --> executor
executor stephen --> executor
executor stephen --> executor
executor william --> executor
executor estate tim --> executor
executor estate uriah --> executor
executors estate jabez --> executors
farmer --> farm

In [1582]:
ExceptionsCT = {'inn keeper' : 'inn keeper', 'leather dresser' : 'leather dresser', 'post rider' : 'post rider', 
                'school committee' : 'school committee', 'school master' : 'school master', 'society committee' : 'committee',
               'society treasurer' : 'treasurer'}

In [1583]:
#Handle duplicate occupations that are spelled differently. Pass in 'occupation' column as a list. Pass in function which will do a manual check for each state (differs w/ each state). 
def MergeOccupations(OccsTotals, Exceptions):
    Occupations = OccsTotals['occupation'].tolist() #Returns 'occupation' column as a list.    
    Index = 0 #Keep track of index. 
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
        
        NewOccupation = Keywords[0][0] #Get new occupation from list of keywords. Based on which keyword has the highest rating.
                
        if (Occupation in Exceptions): #If occupation is an exception, set new occupation. 
            NewOccupation = Exceptions[Occupation]
    
        OccsTotals.loc[Index, 'occupation'] = NewOccupation #Replace occupation with new occupation in dataframe.
        
        Index += 1 #Increment. 
    
    print('After Cleaning')
    print('--------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    OccsTotals = OccsTotals.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations, reset index. 
    
    print('After Removing Duplicates and Summing')
    print('-------------------------------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    Occupations = OccsTotals['occupation'].tolist() #Reinstantiate Occupations after changing OccsTotals
    i = 0 #Declare new index.
    while (i < len(Occupations)): 
        j = 0 #Declare new index.
        while (j < len(Occupations)):                
            if (rapidfuzz.fuzz.ratio(Occupations[i], Occupations[j]) > 80) and (Occupations[i] != Occupations[j]) and ((Occupations[j] not in NoMerge) and (Occupations[i] not in NoMerge))  : #Use fuzzy string matching to determine whether two occupations are the same or not.               
                Total = float(OccsTotals.loc[j, '6p_total']) #Get total from duplicate occupation.
                OccsTotals.loc[i, '6p_total'] += Total #Merge totals together.
                OccsTotals.drop(j, inplace=True) #Remove duplicate occupation.
                OccsTotals = OccsTotals.reset_index(drop=True) #Reset index.
                
                print('Removed: ' + Occupations[j] + ' --> ' + Occupations[i])
                
                Occupations.remove(Occupations[j]) #Remove from list of occupations so loop will not iterate over it again.
            else:
                j += 1
                
        i += 1
    
    print('\n')
    print('After Using Fuzzy String Matching')
    print('---------------------------------')
    print('\n')
    print(OccsTotals.to_string())
    print('\n')
    
    return OccsTotals
        

In [1584]:
OccsTotalsCT = MergeOccupations(OccsTotalsCT, ExceptionsCT)

After Cleaning
--------------


          occupation  6p_total
0      adminastrator    218.47
1      adminastrator      0.00
2      adminastrator     61.30
3      adminastrator     94.67
4      adminastrator     73.50
5      administrator   3413.72
6      administrator     54.05
7       adminstrator     77.61
8       adminstrator     87.74
9         apothecary      7.72
10          attorney   4863.67
11          attorney   6250.93
12        blacksmith    724.71
13            broker    208.55
14         carpenter    175.34
15           cashier     85.30
16          clothier     53.33
17         committee    306.28
18         committee    220.49
19            cooper    124.76
20            doctor    647.09
21           esquire  13020.09
22          executor   1963.62
23          executor      6.02
24          executor    266.67
25          executor      4.70
26          executor   1573.33
27          executor    614.19
28         executors    129.72
29            farmer  36715.82
30     

In [1585]:
def TurnToExcel(DF, State):
    DF.to_excel('./FinishedSpreadsheets/Occupations' + State + '.xlsx')

In [1586]:
TurnToExcel(OccsTotalsCT, State)

# Georgia

In [1587]:
State = 'GA'
AllOccsGA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsGA = AllOccsGA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsGA)

   6p_Dollar    6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0     4586.0     66.0000         2293.0       33.0000     3026.0     11.00   
1        1.0     95.1133            1.0       47.5567        NaN      0.00   
2        1.0     90.3000            1.0       45.1000        1.0     93.50   
3        1.0    129.6533            1.0       64.8233        1.0    144.96   
4        1.0    209.8633            1.0      104.9333        1.0    241.38   
5        1.0  10108.9033            1.0     5054.4533        1.0   2729.40   
6        1.0    114.7330            1.0       57.3330        1.0    124.45   
7       20.0     26.1200           16.0       13.0600        NaN       NaN   
8       37.0     18.2000           18.0        9.0900        NaN      0.00   
9    10508.0     90.0000            NaN           NaN        NaN       NaN   

               town state occupation           county name_type      6p_total  
0  State of Georgia    GA  Gentleman              NaN     sta

In [1588]:
OccsTotalsGA = AllOccsGA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsGA.to_string())

  occupation      6p_total
0        Esq    102.089033
1  Gentleman   4586.660000
2   Merchant  10538.261296
3  Merchants     39.478533


In [1589]:
OccsTotalsGA = RemoveStopwords(OccsTotalsGA) 
print(OccsTotalsGA.to_string())

  occupation      6p_total
0        esq    102.089033
1  gentleman   4586.660000
2   merchant  10538.261296
3  merchants     39.478533


In [1590]:
FindRemoveConst(OccsTotalsGA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


esq --> esq
gentleman --> gentleman
merchant --> merchant
merchants --> merchants


In [1591]:
ExceptionsGA = {}

In [1592]:
OccsTotalsGA = MergeOccupations(OccsTotalsGA, ExceptionsGA)

After Cleaning
--------------


  occupation      6p_total
0        esq    102.089033
1  gentleman   4586.660000
2   merchant  10538.261296
3  merchants     39.478533


After Removing Duplicates and Summing
-------------------------------------


  occupation      6p_total
0        esq    102.089033
1  gentleman   4586.660000
2   merchant  10538.261296
3  merchants     39.478533


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


  occupation      6p_total
0        esq    102.089033
1  gentleman   4586.660000
2   merchant  10577.739829




In [1593]:
TurnToExcel(OccsTotalsGA, State)

# Maryland

In [1594]:
State = 'MD'
AllOccsMD = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsMD = AllOccsMD.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsMD)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        17.0   66.5733           13.0       33.2833       14.0      76.4   
1     26745.0   77.0000        13372.0       88.0000    50000.0      51.0   
2      1243.0   42.0000          621.0       71.0000     1796.0      44.0   
3       117.0   90.0000           58.0       94.0000      113.0      53.0   
4       141.0    3.0000           70.0       51.0000      375.0      32.0   
5        40.0   81.0000           20.0       40.0000       40.0      79.0   
6       789.0   24.0000          394.0       61.0000        NaN       NaN   
7       812.0   33.0000          406.0       16.0000      999.0      45.0   
8       341.0    8.0000          170.0       53.0000      265.0      47.0   
9      1122.0   48.0000          561.0       24.0000      761.0      65.0   
10     1926.0   70.0000          963.0       35.0000      696.0      98.0   
11     3120.0   91.0000         1560.0       45.0000     2499.0      70.0   

In [1595]:
OccsTotalsMD = AllOccsMD.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsMD.to_string())

                         occupation      6p_total
0     Administrator of John Johnson     22.500000
1   Adminstrator of Robert Whitaker      0.000000
2                               Esq     17.665733
3            Executor of Moses Ruth     44.840000
4        Executor of Sabret Sollars    141.030000
5        Executor of Thomas Sollars    117.900000
6                 Frederick Company    206.670000
7                          Merchant  48636.870000
8                         Merchants  29874.070000
9                         Shoemaker   1237.070000
10                       Shopkeeper     33.330000


In [1596]:
OccsTotalsMD = RemoveStopwords(OccsTotalsMD) 
print(OccsTotalsMD.to_string())

                      occupation      6p_total
0     administrator john johnson     22.500000
1   adminstrator robert whitaker      0.000000
2                            esq     17.665733
3            executor moses ruth     44.840000
4        executor sabret sollars    141.030000
5        executor thomas sollars    117.900000
6              frederick company    206.670000
7                       merchant  48636.870000
8                      merchants  29874.070000
9                      shoemaker   1237.070000
10                    shopkeeper     33.330000


In [1597]:
FindRemoveConst(OccsTotalsMD['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administrator john johnson --> administrator
adminstrator robert whitaker --> adminstrator
esq --> esq
executor moses ruth --> executor
executor sabret sollars --> executor
executor thomas sollars --> executor
frederick company --> frederick company
merchant --> merchant
merchants --> merchants
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [1598]:
PrintFutureOccupations(OccsTotalsMD['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator --> administrator
adminstrator --> adminstrator
esq --> esq
executor --> executor
executor --> executor
executor --> executor
frederick company --> frederick
merchant --> merchant
merchants --> merchants
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [1599]:
ExceptionsMD = {'frederick company' : 'company'}

In [1600]:
OccsTotalsMD = MergeOccupations(OccsTotalsMD, ExceptionsMD)

After Cleaning
--------------


       occupation      6p_total
0   administrator     22.500000
1    adminstrator      0.000000
2             esq     17.665733
3        executor     44.840000
4        executor    141.030000
5        executor    117.900000
6         company    206.670000
7        merchant  48636.870000
8       merchants  29874.070000
9       shoemaker   1237.070000
10     shopkeeper     33.330000


After Removing Duplicates and Summing
-------------------------------------


      occupation      6p_total
0  administrator     22.500000
1   adminstrator      0.000000
2        company    206.670000
3            esq     17.665733
4       executor    303.770000
5       merchant  48636.870000
6      merchants  29874.070000
7      shoemaker   1237.070000
8     shopkeeper     33.330000


Removed: adminstrator --> administrator
Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


      occupation      6p_total
0  administrator 

In [1601]:
TurnToExcel(OccsTotalsMD, State)

# North Carolina

In [1602]:
State = 'NC'
AllOccsNC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNC = AllOccsNC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNC)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        85.0      25.0           42.0          62.0       41.0      82.0   
1      2008.0      37.0         1004.0          18.0     1324.0      76.0   
2        48.0      50.0           24.0          25.0        NaN       NaN   
3       176.0       NaN           88.0           NaN      150.0      67.0   
4     12283.0      44.0         6141.0          71.0    10781.0      94.0   
5        41.0      26.0           20.0          64.0       15.0      83.0   
6       206.0       8.0          103.0           5.0       50.0      69.0   
7       228.0      12.0          114.0           6.0      157.0      80.0   
8       252.0      32.0          126.0          16.0        NaN       NaN   
9      4780.0      84.0         2390.0          41.0     4998.0      96.0   
10        0.0       0.0          892.0          66.0     1339.0       NaN   
11       62.0      51.0           31.0          26.0        NaN       NaN   

In [1603]:
OccsTotalsNC = AllOccsNC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsNC.to_string())

                    occupation  6p_total
0      Asse of John B Ashe Exe      0.00
1            Doctors of Physic   2008.37
2                       Farmer     62.51
3                     Merchant    813.03
4                    Merchants     48.50
5  The State of North Carolina   4780.84
6                    Treasurer  12283.44


In [1604]:
OccsTotalsNC = RemoveStopwords(OccsTotalsNC) 
print(OccsTotalsNC.to_string())

                 occupation  6p_total
0      asse john b ashe exe      0.00
1            doctors physic   2008.37
2                    farmer     62.51
3                  merchant    813.03
4                 merchants     48.50
5  the state north carolina   4780.84
6                 treasurer  12283.44


In [1605]:
FindRemoveConst(OccsTotalsNC['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


asse john b ashe exe --> asse john b
doctors physic --> doctors physic
farmer --> farmer
merchant --> merchant
merchants --> merchants
the state north carolina --> the state
treasurer --> treasurer


In [1606]:
Remove = 0

In [1607]:
PrintFutureOccupations(OccsTotalsNC['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


asse john b ashe exe --> asse
doctors physic --> doctors
farmer --> farmer
merchant --> merchant
merchants --> merchants
the state north carolina --> carolina
treasurer --> treasurer


In [1608]:
ExceptionsNC = {'the state north carolina' : 'north carolina'}

In [1609]:
OccsTotalsNC = MergeOccupations(OccsTotalsNC, ExceptionsNC)

After Cleaning
--------------


       occupation  6p_total
0            asse      0.00
1         doctors   2008.37
2          farmer     62.51
3        merchant    813.03
4       merchants     48.50
5  north carolina   4780.84
6       treasurer  12283.44


After Removing Duplicates and Summing
-------------------------------------


       occupation  6p_total
0            asse      0.00
1         doctors   2008.37
2          farmer     62.51
3        merchant    813.03
4       merchants     48.50
5  north carolina   4780.84
6       treasurer  12283.44


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


       occupation  6p_total
0            asse      0.00
1         doctors   2008.37
2          farmer     62.51
3        merchant    861.53
4  north carolina   4780.84
5       treasurer  12283.44




In [1610]:
TurnToExcel(OccsTotalsNC, State)

# New Hampshire

In [1611]:
State = 'NH'
AllOccsNH = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNH = AllOccsNH.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNH)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        200.0       NaN          100.0           NaN       54.0       NaN   
1        778.0       4.0          389.0           2.0      288.0       NaN   
2       1490.0      73.0          745.0          37.0      436.0      77.0   
3       2100.0      48.0         1050.0          23.0      928.0      46.0   
4        149.0      62.0           74.0          81.0        NaN       NaN   
..         ...       ...            ...           ...        ...       ...   
161       81.0      69.0           40.0          84.0      741.0      54.0   
162      187.0       2.0           93.0          51.0      146.0      70.0   
163       12.0      73.0            6.0          37.0        0.0       0.0   
164        9.0      16.0            4.0          58.0       10.0      16.0   
165       40.0      67.0           20.0          33.0       25.0      96.0   

                town state occupation             county name_t

In [1612]:
OccsTotalsNH = AllOccsNH.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsNH.to_string())

    occupation  6p_total
0        Baker   2159.04
1   Blacksmith     54.88
2        Clerk    715.60
3   Cordwainer    742.05
4      Esquire  61539.89
5       Farmer    778.04
6    Gentleman   3603.06
7    Goldsmith    871.32
8       Joiner    106.61
9     Marniner    785.62
10    Merchant  29715.41
11     Painter     48.61
12   Physician   7111.43
13    Spinster     89.13
14      Trader   2807.27
15   Treasurer    149.62
16       Widow   8790.45
17      Yeoman   3397.94


In [1613]:
OccsTotalsNH = RemoveStopwords(OccsTotalsNH) 
print(OccsTotalsNH.to_string())

    occupation  6p_total
0        baker   2159.04
1   blacksmith     54.88
2        clerk    715.60
3   cordwainer    742.05
4      esquire  61539.89
5       farmer    778.04
6    gentleman   3603.06
7    goldsmith    871.32
8       joiner    106.61
9     marniner    785.62
10    merchant  29715.41
11     painter     48.61
12   physician   7111.43
13    spinster     89.13
14      trader   2807.27
15   treasurer    149.62
16       widow   8790.45
17      yeoman   3397.94


In [1614]:
FindRemoveConst(OccsTotalsNH['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [1615]:
PrintFutureOccupations(OccsTotalsNH['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [1616]:
ExceptionsNH = {}

In [1617]:
OccsTotalsNH = MergeOccupations(OccsTotalsNH, ExceptionsNH)

After Cleaning
--------------


    occupation  6p_total
0        baker   2159.04
1   blacksmith     54.88
2        clerk    715.60
3   cordwainer    742.05
4      esquire  61539.89
5       farmer    778.04
6    gentleman   3603.06
7    goldsmith    871.32
8       joiner    106.61
9     marniner    785.62
10    merchant  29715.41
11     painter     48.61
12   physician   7111.43
13    spinster     89.13
14      trader   2807.27
15   treasurer    149.62
16       widow   8790.45
17      yeoman   3397.94


After Removing Duplicates and Summing
-------------------------------------


    occupation  6p_total
0        baker   2159.04
1   blacksmith     54.88
2        clerk    715.60
3   cordwainer    742.05
4      esquire  61539.89
5       farmer    778.04
6    gentleman   3603.06
7    goldsmith    871.32
8       joiner    106.61
9     marniner    785.62
10    merchant  29715.41
11     painter     48.61
12   physician   7111.43
13    spinster     89.13
14      trader   2807.27
15   treasure

In [1618]:
TurnToExcel(OccsTotalsNH, State)

# New Jersey

In [1619]:
State = 'NJ'
AllOccsNJ = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNJ = AllOccsNJ.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNJ)

   6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0     1324.0      81.0          662.0          41.0      675.0      52.0   
1     1325.0      68.0          662.0          85.0      714.0      50.0   
2      384.0      34.0          192.0          65.0      156.0      34.0   
3     5017.0       5.0         2508.0          52.0     2053.0      54.0   
4     1046.0      10.0          523.0           5.0      297.0      74.0   
5      235.0       1.0          117.0          51.0      139.0      10.0   
6        3.0      34.0            1.0          66.0        2.0      90.0   
7      184.0       NaN           92.0           NaN      437.0       6.0   
8       16.0      67.0            8.0          33.0        8.0      57.0   

                         town state  \
0                  New Jersey    NJ   
1                  New Jersey    NJ   
2                  New Jersey    NJ   
3                  New Jersey    NJ   
4                  New Jersey    NJ   
5  Mo

In [1620]:
OccsTotalsNJ = AllOccsNJ.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsNJ.to_string())

                                                   occupation  6p_total
0                                             Attorney at law   1325.68
1                                                     Esquire      0.00
2                        Executor of Michael De Mott deceased   1324.81
3                                                      Farmer      3.34
4                                                   Gentleman   6063.15
5  In trust for Jacobus Van Eis of Somerset County New Jersey     16.67
6                                                    Merchant    235.01
7                                                   Physician    384.34


In [1621]:
OccsTotalsNJ = RemoveStopwords(OccsTotalsNJ) 
print(OccsTotalsNJ.to_string())

                                            occupation  6p_total
0                                         attorney law   1325.68
1                                              esquire      0.00
2                    executor michael de mott deceased   1324.81
3                                               farmer      3.34
4                                            gentleman   6063.15
5  in trust jacobus van eis somerset county new jersey     16.67
6                                             merchant    235.01
7                                            physician    384.34


In [1622]:
FindRemoveConst(OccsTotalsNJ['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


attorney law --> attorney law
esquire --> esquire
executor michael de mott deceased --> executor michael de mott deceased
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county new jersey --> in trust jacobus van eis somerset county new jersey
merchant --> merchant
physician --> physician


In [1623]:
Remove = 2

In [1624]:
PrintFutureOccupations(OccsTotalsNJ['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


attorney law --> attorney
esquire --> esquire
executor michael de --> executor
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county --> county
merchant --> merchant
physician --> physician


In [1625]:
ExceptionsNJ = {'in trust jacobus van eis somerset county' : 'trust'}

In [1626]:
OccsTotalsNJ = MergeOccupations(OccsTotalsNJ, ExceptionsNJ)

After Cleaning
--------------


  occupation  6p_total
0   attorney   1325.68
1    esquire      0.00
2   executor   1324.81
3     farmer      3.34
4  gentleman   6063.15
5      trust     16.67
6   merchant    235.01
7  physician    384.34


After Removing Duplicates and Summing
-------------------------------------


  occupation  6p_total
0   attorney   1325.68
1    esquire      0.00
2   executor   1324.81
3     farmer      3.34
4  gentleman   6063.15
5   merchant    235.01
6  physician    384.34
7      trust     16.67




After Using Fuzzy String Matching
---------------------------------


  occupation  6p_total
0   attorney   1325.68
1    esquire      0.00
2   executor   1324.81
3     farmer      3.34
4  gentleman   6063.15
5   merchant    235.01
6  physician    384.34
7      trust     16.67




In [1627]:
TurnToExcel(OccsTotalsNJ, State)

# New York

In [1628]:
State = 'NY'
AllOccsNY = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNY = AllOccsNY.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNY)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       131.0   15.0000           65.0       57.0000      146.0     69.00   
1       118.0   45.0000           59.0       23.0000      124.0     94.00   
2        23.0   69.0000           11.0       84.0000       24.0     99.00   
3       261.0   31.0000          130.0       65.0000      340.0     53.00   
4       933.0   33.0000          466.0       67.0000      781.0     67.00   
5       555.0   86.0000          277.0       93.0000      637.0     27.00   
6       230.0   90.0000          115.0       45.0000      179.0     91.00   
7       377.0   49.0000          188.0       75.0000      385.0     98.00   
8       286.0   43.0000          143.0       22.0000      248.0     68.00   
9         NaN    1.0000            NaN           NaN        NaN       NaN   
10        NaN       NaN            NaN           NaN     1601.0     94.00   
11        1.0   46.7633            1.0       23.3833        1.0     63.34   

In [1629]:
OccsTotalsNY = AllOccsNY.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsNY.to_string())

     occupation     6p_total
0        Broker  1776.120000
1        Farmer   674.310000
2       Mariner   377.490000
3      Merchant  4997.975500
4     Merchants  2450.827633
5     Physician   933.330000
6  Schoolmaster   131.150000


In [1630]:
OccsTotalsNY = RemoveStopwords(OccsTotalsNY) 
print(OccsTotalsNY.to_string())

     occupation     6p_total
0        broker  1776.120000
1        farmer   674.310000
2       mariner   377.490000
3      merchant  4997.975500
4     merchants  2450.827633
5     physician   933.330000
6  schoolmaster   131.150000


In [1631]:
PrintFutureOccupations(OccsTotalsNY['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


broker --> broker
farmer --> farmer
mariner --> mariner
merchant --> merchant
merchants --> merchants
physician --> physician
schoolmaster --> schoolmaster


In [1632]:
ExceptionsNY = {}

In [1633]:
OccsTotalsNY = MergeOccupations(OccsTotalsNY, ExceptionsNY)

After Cleaning
--------------


     occupation     6p_total
0        broker  1776.120000
1        farmer   674.310000
2       mariner   377.490000
3      merchant  4997.975500
4     merchants  2450.827633
5     physician   933.330000
6  schoolmaster   131.150000


After Removing Duplicates and Summing
-------------------------------------


     occupation     6p_total
0        broker  1776.120000
1        farmer   674.310000
2       mariner   377.490000
3      merchant  4997.975500
4     merchants  2450.827633
5     physician   933.330000
6  schoolmaster   131.150000


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


     occupation     6p_total
0        broker  1776.120000
1        farmer   674.310000
2       mariner   377.490000
3      merchant  7448.803133
4     physician   933.330000
5  schoolmaster   131.150000




In [1634]:
TurnToExcel(OccsTotalsNY, State)

# Pennsylvania

In [1635]:
State = 'PA'
AllOccsPA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsPA = AllOccsPA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsPA)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0    102.14            1.0         51.07        1.0    117.34   
1       3212.0     38.00         1606.0         19.00     1400.0     66.00   
2       4529.0     96.00         2264.0         98.00     3312.0     47.00   
3        607.0      8.00          303.0         54.00      146.0     51.00   
4       2050.0     74.00         1025.0         37.00      745.0     57.00   
..         ...       ...            ...           ...        ...       ...   
410     9544.0     97.00         4772.0         48.00     8590.0     46.00   
411      880.0     75.00          440.0         38.00      753.0     91.00   
412       20.0     25.00           10.0         13.00        NaN       NaN   
413       43.0     60.00           21.0         80.00        NaN       NaN   
414      150.0       NaN           75.0           NaN      138.0     90.00   

                         town state occupation               co

In [1636]:
OccsTotalsPA = AllOccsPA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsPA.to_string())

                                                         occupation     6p_total
0                                                        AH.Vat Law      58.0500
1        Adm of Alexander McCaraher late of Chester County Deceased     799.5100
2                            Administer of the estate of Job Vernon    1865.4400
3                                  Administor to the Est of J James      27.5300
4                                                   Attorney at Law    1223.6200
5                                                       Black Smith     692.2500
6                                                       Book Binder     783.0700
7                                                            Broker   64008.8400
8                                                             Clerk    1855.5800
9                                                                Co    2898.8600
10                                                      Conveyancer     108.4000
11                          

In [1637]:
OccsTotalsPA = RemoveStopwords(OccsTotalsPA) 
print(OccsTotalsPA.to_string())

                                               occupation     6p_total
0                                              ah.vat law      58.0500
1    adm alexander mccaraher late chester county deceased     799.5100
2                            administer estate job vernon    1865.4400
3                                  administor est j james      27.5300
4                                            attorney law    1223.6200
5                                             black smith     692.2500
6                                             book binder     783.0700
7                                                  broker   64008.8400
8                                                   clerk    1855.5800
9                                                      co    2898.8600
10                                            conveyancer     108.4000
11                                            cordwainder     281.2500
12                                               druggist    4922.4300
13    

In [1638]:
FindRemoveConst(OccsTotalsPA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


ah.vat law --> ah.vat law
adm alexander mccaraher late chester county deceased --> adm alexander mccaraher late chester
administer estate job vernon --> administer estate
administor est j james --> administor est
attorney law --> attorney law
black smith --> black smith
book binder --> book binder
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john benezet deceased --> estate john
estate --> estate
ex philadelphia county --> ex
ex william honeyman --> ex
ex estate john baird bucks co --> ex estate john baird
executers david grierlate y town --> executers david grierlate
executor estate abraham hull deceased --> executor estate abraham
executor estate mathias londenberger --> executor estate
executor estate doctor benjamin franklin deceased --> executor estate doctor benjamin
farmer --> f

In [1639]:
PrintFutureOccupations(OccsTotalsPA['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


ah.vat law --> ah.vat
adm alexander mccaraher late chester --> adm
administer estate --> administer
administor est --> administor
attorney law --> attorney
black smith --> black
book binder --> book
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john --> estate
estate --> estate
ex --> ex
ex --> ex
ex estate john baird --> baird
executers david grierlate --> executers
executor estate abraham --> executor
executor estate --> executor
executor estate doctor benjamin --> executor
farmer --> farmer
furrier --> furrier
gentleman --> gentleman
goldsmith --> goldsmith
guardian --> guardian
guardian --> guardian
guardian --> guardian
hatter --> hatter
inn keeper --> inn
junior broker --> junior
law --> law
lawyer --> lawyer
marchant --> marchant
mariner --> mariner
merchant --> mer

In [1640]:
ExceptionsPA = {'ah.vat law' : 'attorney', 'black smith' : 'blacksmith', 'book binder' : 'book binder', 'ex estate john baird' : 'executors', 'ex' : 'executors',
               'inn keeper' : 'inn keeper', 'junior broker' : 'broker', 'post master' : 'post master', 'ship wright' : 'ship wright', 'shop keeper' : 'shopkeeper', 
               'christopher mary' : 'individual', 'law' : 'attorney', 'lawyer' : 'attorney', 'adm alexander mccaraher late chester' : 'administer'}

In [1641]:
OccsTotalsPA = MergeOccupations(OccsTotalsPA, ExceptionsPA)

After Cleaning
--------------


      occupation     6p_total
0       attorney      58.0500
1     administer     799.5100
2     administer    1865.4400
3     administor      27.5300
4       attorney    1223.6200
5     blacksmith     692.2500
6    book binder     783.0700
7         broker   64008.8400
8          clerk    1855.5800
9             co    2898.8600
10   conveyancer     108.4000
11   cordwainder     281.2500
12      druggist    4922.4300
13       esquier   20672.0700
14       esquire   20770.1700
15        estate       0.0000
16        estate    1979.6000
17     executors    1802.9600
18     executors     668.9600
19     executors     115.9400
20     executers    5658.3600
21      executor      47.9100
22      executor     493.8300
23      executor       0.0000
24        farmer    1685.8100
25       furrier    3107.1800
26     gentleman    7540.6600
27     goldsmith     142.2200
28      guardian    1605.8000
29      guardian    2267.5000
30      guardian     666.6600
31      

In [1642]:
TurnToExcel(OccsTotalsPA, State)

# Rhode Island

In [1643]:
State = 'RI'
AllOccsRI = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsRI = AllOccsRI.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsRI)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        154.0      20.0           77.0          10.0      192.0       NaN   
1        194.0      54.0           97.0          27.0      222.0       1.0   
2        359.0      50.0          179.0          76.0      254.0      21.0   
3          7.0      16.0            3.0          58.0       55.0      82.0   
4         35.0       2.0           17.0          51.0       38.0      94.0   
..         ...       ...            ...           ...        ...       ...   
443      164.0      96.0          165.0          27.0      100.0      22.0   
444       59.0      90.0           29.0          95.0       70.0       8.0   
445       28.0      44.0           14.0          22.0       29.0      70.0   
446      142.0      44.0           71.0          22.0      132.0      43.0   
447       33.0      33.0           17.0          39.0       31.0      94.0   

             town state occupation             county name_type

In [1644]:
OccsTotalsRI = AllOccsRI.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsRI.to_string())

                                                                          occupation   6p_total
0                      Administratrix to the Estate of Joshua Brown Esquire Deceased       0.00
1                                                                        Auctioneers     460.77
2                                                                              Baker     206.37
3                                                                         Blacksmith     540.68
4                                                                         Blockmaker       4.43
5                                                                             Broker     171.50
6                                                                            Butcher     235.70
7                                                                          Carpenter       0.00
8                                                                       Chaise Maker      47.57
9                                       

In [1645]:
OccsTotalsRI = RemoveStopwords(OccsTotalsRI) 
print(OccsTotalsRI.to_string())

                                                       occupation   6p_total
0             administratrix estate joshua brown esquire deceased       0.00
1                                                     auctioneers     460.77
2                                                           baker     206.37
3                                                      blacksmith     540.68
4                                                      blockmaker       4.43
5                                                          broker     171.50
6                                                         butcher     235.70
7                                                       carpenter       0.00
8                                                    chaise maker      47.57
9                                                          cooper    1596.15
10                                                     cordwainer     417.48
11                                                         deacon      16.35

In [1646]:
FindRemoveConst(OccsTotalsRI['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administratrix estate joshua brown esquire deceased --> administratrix estate joshua brown
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise maker
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq deceased --> extors late jn jenckes
extors late nicholas brown esq deceased --> extors late nicholas brown
farmer --> farmer
general treasurer state rhode island suceessors said office --> general treasurer state rhode island suceessors
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> m

In [1647]:
Remove = 1

In [1648]:
PrintFutureOccupations(OccsTotalsRI['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administratrix estate joshua brown esquire --> administratrix
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq --> extors
extors late nicholas brown esq --> extors
farmer --> farmer
general treasurer state rhode island suceessors said --> general
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> merchants
physician --> physician
reverend --> reverend
shipwright --> shipwright
shoemaker --> shoemaker
spinster --> spin

In [1649]:
ExceptionsRI = {'general treasurer state rhode island suceessors said' : 'treasurer', 'behalf proprietors' : 'proprietors', 
                'chaise maker' : 'chaise maker'}

In [1650]:
OccsTotalsRI = MergeOccupations(OccsTotalsRI, ExceptionsRI)

After Cleaning
--------------


        occupation   6p_total
0   administratrix       0.00
1      auctioneers     460.77
2            baker     206.37
3       blacksmith     540.68
4       blockmaker       4.43
5           broker     171.50
6          butcher     235.70
7        carpenter       0.00
8     chaise maker      47.57
9           cooper    1596.15
10      cordwainer     417.48
11          deacon      16.35
12       distiller     466.25
13          doctor     445.45
14         esquier      13.08
15         esquire   68388.42
16          extors     124.50
17          extors       0.00
18          farmer    1487.53
19       treasurer  199740.67
20       gentleman    2396.70
21         glazier     982.35
22       goldsmith       0.00
23          hatter     828.66
24     housewright     266.67
25       innholder    1430.29
26        labourer      67.16
27         mariner    4857.32
28           mason    1152.50
29        merchant   99957.63
30       merchants       4.29
31      

In [1651]:
TurnToExcel(OccsTotalsRI, State)

# South Carolina

In [1652]:
State = 'SC'
AllOccsSC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsSC = AllOccsSC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsSC)

     6p_Dollar   6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0   824.4133            1.0      412.2033        1.0    941.54   
1          1.0  3987.9033            1.0     1993.9533        1.0   3625.67   
2         17.0   150.0000           13.0       75.0000       14.0    129.10   
3         17.0  1067.9200           13.0      533.9600       14.0   1165.92   
4         17.0   195.0200           13.0       97.5100       14.0    212.82   
..         ...        ...            ...           ...        ...       ...   
286        NaN        NaN            NaN           NaN      651.0     53.00   
287        NaN        NaN            NaN           NaN      140.0     77.00   
288        NaN        NaN            NaN           NaN      167.0     77.00   
289        NaN        NaN            NaN           NaN     3600.0       NaN   
290      180.0    47.0000           90.0       24.0000      207.0     93.00   

               town state       occupation         

In [1653]:
OccsTotalsSC = AllOccsSC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsSC.to_string())

                                      occupation      6p_total
0            Administrator to John Harleston Jun     33.330000
1                 Administrator to Joseph Warley   1671.440000
2                       Assignee of James Simons    345.020000
3                                Attorney at Law   3690.940000
4                                      Autioneer     10.650000
5                                   Blacks smith   1423.480000
6                                     Blockmaker   5264.230000
7                                     Bricklayer     37.350000
8                                         Broker   2334.890000
9                                  Cabinet Maker     74.460000
10                                     Carpenter    449.020000
11                          Charleston Merchants     26.670000
12                           Clerk of the Senate   1425.820000
13                     Comtroller of the Customs     53.740000
14                                           Esq     50

In [1654]:
OccsTotalsSC = RemoveStopwords(OccsTotalsSC) 
print(OccsTotalsSC.to_string())

                           occupation      6p_total
0    administrator john harleston jun     33.330000
1         administrator joseph warley   1671.440000
2               assignee james simons    345.020000
3                        attorney law   3690.940000
4                           autioneer     10.650000
5                        blacks smith   1423.480000
6                          blockmaker   5264.230000
7                          bricklayer     37.350000
8                              broker   2334.890000
9                       cabinet maker     74.460000
10                          carpenter    449.020000
11               charleston merchants     26.670000
12                       clerk senate   1425.820000
13                 comtroller customs     53.740000
14                                esq     50.123166
15                           excuetor     18.950200
16            executor philip hawkins    239.100000
17           executor benjamin tucker    834.860000
18          

In [1655]:
FindRemoveConst(OccsTotalsSC['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administrator john harleston jun --> administrator john harleston
administrator joseph warley --> administrator joseph
assignee james simons --> assignee james
attorney law --> attorney law
autioneer --> autioneer
blacks smith --> blacks smith
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet maker
carpenter --> carpenter
charleston merchants --> charleston merchants
clerk senate --> clerk senate
comtroller customs --> comtroller customs
esq --> esq
excuetor --> excuetor
executor philip hawkins --> executor philip
executor benjamin tucker --> executor benjamin
executor jacob smiser --> executor jacob
executor james peart --> executor james
executor samuel bonneau --> executor samuel
executor sarah lampriere --> executor sarah
executor william gibbs --> executor william
executor william parker --> executor william
executor richard mason --> executor richard
execu

In [1656]:
Remove = 1

In [1657]:
PrintFutureOccupations(OccsTotalsSC['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator john harleston --> administrator
administrator joseph --> administrator
assignee james --> assignee
attorney law --> attorney
autioneer --> autioneer
blacks smith --> blacks
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet
carpenter --> carpenter
charleston merchants --> charleston
clerk senate --> clerk
comtroller customs --> comtroller
esq --> esq
excuetor --> excuetor
executor philip --> executor
executor benjamin --> executor
executor jacob --> executor
executor james --> executor
executor samuel --> executor
executor sarah --> executor
executor william --> executor
executor william --> executor
executor richard --> executor
executrix william --> executrix
factor --> factor
gentleman --> gentleman
hair dresser --> hair
mariner --> mariner
marriner --> marriner
merchant --> merchant
merchant executor benjamin --> merchant
merchant

In [1658]:
ExceptionsSC = {'executrix william' : 'executor', 'blacks smith' : 'blacksmith', 'cabinet maker' : 'cabinet maker', 'charleston merchants' : 'merchants', 
               'hair dresser' : 'hair dresser', 'south carolina' : 'south carolina', 'vendue master' : 'auctioneer'}

In [1659]:
OccsTotalsSC = MergeOccupations(OccsTotalsSC, ExceptionsSC)

After Cleaning
--------------


        occupation      6p_total
0    administrator     33.330000
1    administrator   1671.440000
2         assignee    345.020000
3         attorney   3690.940000
4        autioneer     10.650000
5       blacksmith   1423.480000
6       blockmaker   5264.230000
7       bricklayer     37.350000
8           broker   2334.890000
9    cabinet maker     74.460000
10       carpenter    449.020000
11       merchants     26.670000
12           clerk   1425.820000
13      comtroller     53.740000
14             esq     50.123166
15        excuetor     18.950200
16        executor    239.100000
17        executor    834.860000
18        executor     69.020000
19        executor     12.110000
20        executor     23.720000
21        executor     68.260000
22        executor     66.660000
23        executor    685.770000
24        executor   1024.910000
25        executor     76.700000
26          factor   2961.480000
27       gentleman    474.430000
28    hair 

In [1660]:
TurnToExcel(OccsTotalsSC, State)

# Virginia

In [1661]:
State = 'VA'
AllOccsVA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsVA = AllOccsVA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsVA)

   6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       17.0  351.6233           13.0      175.8133       14.0    384.09   
1       78.0   33.0000           39.0       17.0000       42.0     30.00   
2       66.0   66.0000           33.0       33.0000       63.0     24.00   
3        3.0   82.0000            1.0       92.0000        3.0     74.00   
4       46.0    2.0000           23.0        1.0000       44.0     89.00   

                        town state occupation             county name_type  \
0        Portsmouth Virginia    VA  Merchants  Rockingham County      town   
1   Accomack County Virginia    VA  Gentleman    Accomack County    county   
2  Virginia and Philadelphia    VA   Merchant                NaN     state   
3  Virginia and Philadelphia    VA   Merchant                NaN     state   
4              Williamsburgh    VA     Weaver  Greenbrier County      town   

    6p_total  
0  20.516233  
1  78.330000  
2  66.660000  
3   3.820000  

In [1662]:
OccsTotalsVA = AllOccsVA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsVA.to_string())

  occupation   6p_total
0  Gentleman  78.330000
1   Merchant  70.480000
2  Merchants  20.516233
3     Weaver  46.020000


In [1663]:
OccsTotalsVA = RemoveStopwords(OccsTotalsVA) 
print(OccsTotalsVA.to_string())

  occupation   6p_total
0  gentleman  78.330000
1   merchant  70.480000
2  merchants  20.516233
3     weaver  46.020000


In [1664]:
FindRemoveConst(OccsTotalsVA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
weaver --> weaver


In [1665]:
PrintFutureOccupations(OccsTotalsVA['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
weaver --> weaver


In [1666]:
ExceptionsVA = {}

In [1667]:
OccsTotalsVA = MergeOccupations(OccsTotalsVA, ExceptionsVA)

After Cleaning
--------------


  occupation   6p_total
0  gentleman  78.330000
1   merchant  70.480000
2  merchants  20.516233
3     weaver  46.020000


After Removing Duplicates and Summing
-------------------------------------


  occupation   6p_total
0  gentleman  78.330000
1   merchant  70.480000
2  merchants  20.516233
3     weaver  46.020000


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


  occupation   6p_total
0  gentleman  78.330000
1   merchant  90.996233
2     weaver  46.020000




In [1668]:
TurnToExcel(OccsTotalsVA, State)

# All States

In [1669]:
#Append each states occupation + totals dataframes to the end of each other. 
OccsTotalsAll = pd.concat([OccsTotalsCT, OccsTotalsGA, OccsTotalsMD, OccsTotalsNC, OccsTotalsNH, OccsTotalsNJ,
                          OccsTotalsNY, OccsTotalsPA, OccsTotalsRI, OccsTotalsSC, OccsTotalsVA]).reset_index(drop=True)
print(OccsTotalsAll.to_string())

           occupation       6p_total
0       adminastrator    4081.060000
1          apothecary       7.720000
2            attorney   11114.600000
3          blacksmith     724.710000
4              broker     208.550000
5           carpenter     175.340000
6             cashier      85.300000
7            clothier      53.330000
8           committee     863.410000
9              cooper     124.760000
10             doctor     647.090000
11            esquire   13020.090000
12           executor    4558.250000
13             farmer   36715.820000
14          gentleman   16624.320000
15          goldsmith       6.820000
16           guardian    1245.620000
17             hatter     242.990000
18         inn keeper     201.530000
19             joiner      53.330000
20    leather dresser     666.670000
21            mariner    3831.050000
22           merchant   63765.000000
23             miller      18.990000
24            molster     211.380000
25          physician    1303.460000
2

In [1670]:
OccsTotalsAll = OccsTotalsAll.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsAll.to_string())

           occupation       6p_total
0       adminastrator    4081.060000
1          administer    2692.480000
2       administrator    1727.270000
3      administratrix       0.000000
4          apothecary       7.720000
5                asse       0.000000
6            assignee     345.020000
7            attorney   24827.900000
8          auctioneer     123.190000
9         auctioneers     460.770000
10              baker    2365.410000
11         blacksmith    3436.000000
12         blockmaker    5268.660000
13        book binder     783.070000
14         bricklayer      37.350000
15             broker   79191.020000
16            butcher     235.700000
17      cabinet maker      74.460000
18          carpenter     624.360000
19            cashier      85.300000
20       chaise maker      47.570000
21              clerk    3997.000000
22           clothier      53.330000
23                 co    2898.860000
24          committee     863.410000
25            company     206.670000
2

In [1671]:
PrintFutureOccupations(OccsTotalsAll['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
administer --> administer
administrator --> administrator
administratrix --> administratrix
apothecary --> apothecary
asse --> asse
assignee --> assignee
attorney --> attorney
auctioneer --> auctioneer
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
book binder --> book
bricklayer --> bricklayer
broker --> broker
butcher --> butcher
cabinet maker --> cabinet
carpenter --> carpenter
cashier --> cashier
chaise maker --> chaise
clerk --> clerk
clothier --> clothier
co --> co
committee --> committee
company --> company
comtroller --> comtroller
conveyancer --> conveyancer
cooper --> cooper
cordwainder --> cordwainder
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
doctors --> doctors
druggist --> druggist
esq --> esq
esquier --> esquier
esquire --> esquire
estate --> estate
excueto

In [1672]:
ExceptionsAll = {'book binder' : 'book binder', 'cabinet maker' : 'cabinet maker', 'chaise maker' : 'chaise maker', 'esq' : 'esquire',
                'extors' : 'executors', 'hair dresser' : 'hair dresser', 'inn keeper' : 'inn keeper', 'junior broker' : 'broker',
                'leather dresser' : 'leather dresser', 'north carolina' : 'north carolina', 'post master' : 'post master', 
                'post rider' : 'post rider', 'school committee' : 'school committee', 'school master' : 'schoolmaster', 'ship wright' : 'shipwright',
                'south carolina' : 'south carolina', 'administer' : 'adminastrator', 'excuetor' : 'executor'}

In [1673]:
NoMerge = ['planter', 'printer']

In [1674]:
OccsTotalsAll = MergeOccupations(OccsTotalsAll, ExceptionsAll)

After Cleaning
--------------


           occupation       6p_total
0       adminastrator    4081.060000
1       adminastrator    2692.480000
2       administrator    1727.270000
3      administratrix       0.000000
4          apothecary       7.720000
5                asse       0.000000
6            assignee     345.020000
7            attorney   24827.900000
8          auctioneer     123.190000
9         auctioneers     460.770000
10              baker    2365.410000
11         blacksmith    3436.000000
12         blockmaker    5268.660000
13        book binder     783.070000
14         bricklayer      37.350000
15             broker   79191.020000
16            butcher     235.700000
17      cabinet maker      74.460000
18          carpenter     624.360000
19            cashier      85.300000
20       chaise maker      47.570000
21              clerk    3997.000000
22           clothier      53.330000
23                 co    2898.860000
24          committee     863.410000
25    

In [1675]:
#Remove any occupatins will a total of '0'
OccsTotalsAll = OccsTotalsAll[OccsTotalsAll['6p_total'] != 0].reset_index(drop=True)
print(OccsTotalsAll.to_string())

          occupation       6p_total
0      adminastrator    8500.810000
1         apothecary       7.720000
2           assignee     345.020000
3           attorney   24827.900000
4         auctioneer     583.960000
5              baker    2365.410000
6         blacksmith    3436.000000
7         blockmaker    5268.660000
8        book binder     783.070000
9         bricklayer      37.350000
10            broker   79191.020000
11           butcher     235.700000
12     cabinet maker      74.460000
13         carpenter     624.360000
14           cashier      85.300000
15      chaise maker      47.570000
16             clerk    3997.000000
17          clothier      53.330000
18                co    2898.860000
19         committee     863.410000
20           company     206.670000
21        comtroller      53.740000
22       conveyancer     108.400000
23            cooper    1720.910000
24       cordwainder    1440.780000
25            deacon      16.350000
26         distiller     466

In [1676]:
TurnToExcel(OccsTotalsAll, 'AllStates')